In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from quantization import *
from train_quantization import init_seeds

2025-03-17 14:49:11.681208: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-17 14:49:11.692861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742237351.707157 1646869 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742237351.711830 1646869 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-17 14:49:11.728128: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [14]:
init_seeds(0)

model = nn.Linear(4, 1)
qt_model = QuantizedWrapper(model, quant_w=True, quant_x=True, lsq=1, 
                            bitwidth_w=4, intbit_w=0, bitwidth_x=4, intbit_x=0, alpha_init=2)
qt_model.g_w = 1.
optimizer = optim.SGD(qt_model.parameters(), lr = 1)

print('--------before--------')
for name, param in qt_model.named_parameters():
    # if name == 'alpha_x' or name == 'alpha_w':
    print(f'name: {name}, param: {param.data}')

qt_model.zero_grad()
x = torch.tensor((10000,10000,10000,10000)).view(1, 4)
y = qt_model(x)
y.backward()
optimizer.step()

print('--------after--------')
print(f'y: {y}')

for name, param in qt_model.named_parameters():
    print(f'name: {name}, param: {param.data}')

for name, param in qt_model.named_parameters():
    # if name == 'alpha_x' or name == 'alpha_w':
    print(f'name: {name}, param: {param.grad}')

--------before--------
name: full_precision_weight, param: tensor([[-0.0037,  0.2682, -0.4115, -0.3680]])
name: alpha_w, param: 0.156248539686203
name: beta_w, param: -0.1287534236907959
name: alpha_x, param: 0.3171935975551605
name: beta_x, param: 0.419644832611084
name: module.weight, param: tensor([[-0.0037,  0.2682, -0.4115, -0.3680]])
name: module.bias, param: tensor([-0.1926])
--------after--------
y: tensor([[-5150.3291]], grad_fn=<AddmmBackward0>)
name: full_precision_weight, param: tensor([[-10000.0039,  -9999.7314, -10000.4111, -10000.3682]])
name: alpha_w, param: 0.155760258436203
name: beta_w, param: -0.1287534236907959
name: alpha_x, param: 0.3171935975551605
name: beta_x, param: 0.419644832611084
name: module.weight, param: tensor([[-10000.0039,  -9999.7314, -10000.4111, -10000.3682]])
name: module.bias, param: tensor([-1.1926])
name: full_precision_weight, param: tensor([[10000., 10000., 10000., 10000.]])
name: alpha_w, param: 0.00048828125
name: beta_w, param: 0.0
name:

In [15]:
init_seeds(0)

model = nn.Linear(4, 1)
qt_model = QuantizedWrapper(model, quant_w=True, quant_x=True, lsq=2, 
                            bitwidth_w=4, intbit_w=0, bitwidth_x=4, intbit_x=0, alpha_init=2)
optimizer = optim.SGD(qt_model.parameters(), lr = 1)

print('--------before--------')
for name, param in qt_model.named_parameters():
    print(f'name: {name}, param: {param.data}')

qt_model.zero_grad()
x = torch.tensor((10000,10000,10000,10000)).view(1, 4)
y = qt_model(x)
y.backward()
optimizer.step()

print('--------after--------')
print(f'y: {y}')
for name, param in qt_model.named_parameters():
    print(f'name: {name}, param: {param.data}')

for name, param in qt_model.named_parameters():
    print(f'name: {name}, param: {param.grad}')

--------before--------
name: full_precision_weight, param: tensor([[-0.0037,  0.2682, -0.4115, -0.3680]])
name: alpha_w, param: 0.156248539686203
name: beta_w, param: -0.1287534236907959
name: alpha_x, param: 0.3171935975551605
name: beta_x, param: 0.419644832611084
name: module.weight, param: tensor([[-0.0037,  0.2682, -0.4115, -0.3680]])
name: module.bias, param: tensor([-0.1926])
--------after--------
y: tensor([[-5150.3291]], grad_fn=<AddmmBackward0>)
name: full_precision_weight, param: tensor([[-10000.0039,  -9999.7314, -10000.4111, -10000.3682]])
name: alpha_w, param: 0.156248539686203
name: beta_w, param: -0.1287534236907959
name: alpha_x, param: 0.3171935975551605
name: beta_x, param: 0.419644832611084
name: module.weight, param: tensor([[-10000.0039,  -9999.7314, -10000.4111, -10000.3682]])
name: module.bias, param: tensor([-1.1926])
name: full_precision_weight, param: tensor([[10000., 10000., 10000., 10000.]])
name: alpha_w, param: 0.0
name: beta_w, param: 0.0
name: alpha_x, 

In [4]:
xv, alpha_wv, beta_wv =  0.2682, 0.156248539686203, -0.1287534236907959

In [5]:
x = torch.tensor([xv], requires_grad=True)
alpha_w = torch.tensor([alpha_wv], requires_grad=True)
beta_w = torch.tensor([beta_wv], requires_grad=True)

print(x.grad)
print(alpha_w.grad)
print(beta_w.grad)
y2 = alpha_w*STEWrapper.apply((x-beta_w)/alpha_w).clamp(-8, 7)+beta_w
y2.backward()
print(y2)
print(x.grad)
print(alpha_w.grad)
print(beta_w.grad)

None
None
None
tensor([0.3400], grad_fn=<AddBackward0>)
tensor([1.])
tensor([0.4595])
tensor([0.])


In [6]:
x = torch.tensor([xv], requires_grad=True)
alpha_w = torch.tensor([alpha_wv], requires_grad=True)
beta_w = torch.tensor([beta_wv], requires_grad=True)
g_w = torch.tensor([1.])

print(x.grad)
print(alpha_w.grad)
print(beta_w.grad)
y1 = LSQuant.apply(x, alpha_w, beta_w, g_w, -8, 7) 
y1.backward()
print(y1)
print(x.grad)
print(alpha_w.grad)
print(beta_w.grad)

None
None
None
tensor([0.3400], grad_fn=<LSQuantBackward>)
tensor([1.])
tensor([0.4595])
tensor([0.])
